In [1]:
def get_backend(device):
    """Returns backend object for device specified by input string."""
    try:
        backend = Aer.get_backend(device)
    except:
        backend = IBMQ.get_backend(device)
    return backend

In [2]:
def get_noise(noisy):
    """Returns a noise model when input is True"""
    if noisy:
        
        if type(noisy) is float:
            p_meas = noisy
            p_gate1 = noisy
        else:
            p_meas = 0.08
            p_gate1 = 0.04

        error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
        error_gate1 = depolarizing_error(p_gate1, 1)
        error_gate2 = error_gate1.kron(error_gate1)
        
        noise_model = NoiseModel()
        noise_model.add_all_qubit_quantum_error(error_meas, "measure")
        noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
        noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"])
    else:
        noise_model = None
    return noise_model

In [3]:
from qiskit import IBMQ
IBMQ.load_accounts()

In [4]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, Aer, IBMQ, compile
import copy
import numpy as np

class random_grid ():
    
        def __init__(self,Lx,Ly):
        
            self.Lx = Lx
            self.Ly = Ly
        
            self.qr = QuantumRegister(Lx*Ly)
            self.cr = ClassicalRegister(Lx*Ly)
            self.qc = QuantumCircuit(self.qr,self.cr)
            
        def _address(self,x,y):
            return y*self.Lx + x
        
        def neighbours(self,x,y):
            neighbours = []
            for (xx,yy) in [(x+1,y),(x-1,y),(x,y+1),(x,y-1)]:
                if (xx>=0) and (xx<=self.Lx-1) and (yy>=0) and (yy<=self.Ly-1):
                    neighbours.append( (xx,yy) )
            return neighbours
                
        def get_samples(self,device='qasm_simulator',noisy=False,shots=1024):
            
            def separate_string(string):
                string = string[::-1]
                grid = []
                for y in range(self.Ly):
                    line = ''
                    for x in range(self.Lx):
                        line += string[self._address(x,y)]
                    grid.append(line)
                return '\n'.join(grid)
            
            temp_qc = copy.deepcopy(self.qc)
            temp_qc.barrier(self.qr)
            temp_qc.measure(self.qr,self.cr)
            try:
                job = execute(temp_qc,backend=get_backend(device),noise_model=get_noise(noisy),shots=shots,memory=True)
            except:
                job = execute(temp_qc,backend=get_backend(device),shots=shots,memory=True)
            
            try:
                data = job.result().get_memory()
                grid_data = []
                for string in data:
                    grid_data.append(separate_string(string))
            except:
                grid_data = None
                
            stats = job.result().get_counts()
            grid_stats = {}
            for string in stats:
                grid_stats[separate_string(string)] = stats[string]
                
            return grid_stats, grid_data
        
        def NOT (self,coords,frac=1,axis='x'):
            '''Implement an rx or ry on the qubit for the given coords, according to the given fraction (`frac=1` is a NOT gate) and the given axis ('x' or 'y').'''
            if axis=='x':
                self.qc.rx(np.pi*frac,self.qr[self._address(coords[0],coords[1])])
            else:
                self.qc.ry(np.pi*frac,self.qr[self._address(coords[0],coords[1])])
            
        def CNOT (self,ctl,tgt,frac=1,axis='x'):
            '''Controlled version of the `NOT` above'''
            if axis=='y':
                self.qc.sdg(self.qr[self._address(tgt[0],tgt[1])])
            self.qc.h(self.qr[self._address(tgt[0],tgt[1])])
            self.qc.crz(np.pi*frac,self.qr[self._address(ctl[0],ctl[1])],self.qr[self._address(tgt[0],tgt[1])])
            self.qc.h(self.qr[self._address(tgt[0],tgt[1])])
            if axis=='y':
                self.qc.s(self.qr[self._address(tgt[0],tgt[1])])
    

In [5]:
grid = random_grid(2,2)
grid.NOT((1,1),frac=1)
grid.CNOT((1,1),(0,0),frac=0.5)

grid_stats,grid_data = grid.get_samples(shots=5)#,device='ibmqx4')
for sample in grid_data:
    print(sample,'\n')

00
01 

10
01 

00
01 

00
01 

10
01 

